In [1]:
from config import *
import dataloader as dl
from models import RNNModel
from training import *

from importlib import reload

import torch
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import time

import pickle as pkl
import numpy as np
import lyapunov_test as lyap

import matplotlib.pyplot as plt

device = torch.device('cuda')

Using TensorFlow backend.


# Data Preprocessing

In [2]:
dcon = DataConfig('data', input_seq_length = 100, target_seq_length = 1, train_frac = 0.8, val_frac = 0.1, test_frac = 0.1, batch_size = 128)    

Loading  data/data.pkl ...
Loading  data/vocab.pkl ...
None
Cutting off end of data so that it divides evenly
Data load done! Number of data batches in train: 200, val: 25, test: 24


# Define Model and Training Configuration Parameters

In [3]:
hidden_size = 64
learning_rate = 0.002
dropout = 0.1
uni_param = 0.08
batch_size = 128
model_type = 'gru'

In [4]:
sch = torch.optim.lr_scheduler.MultiStepLR
sch_params = {'milestones': range(10, 50), 'gamma': 0.95}
max_epoch = 15

In [5]:
mcon = ModelConfig(model_type, 1, hidden_size, dcon.input_size, dropout, 'uni', {'a': -uni_param, 'b':uni_param}, device, bias = False, id_init_param = 'b')
tcon = TrainConfig('Models', batch_size, max_epoch, 'adam', learning_rate, {}, scheduler = sch, scheduler_params= sch_params, start_epoch = 0)
fcon = FullConfig(dcon, tcon, mcon)

In [6]:
model = RNNModel(mcon).to(mcon.device)
optimizer = tcon.get_optimizer(model.parameters())

# Train Models Using Hyperopt Package

In [7]:
def objective(space):
    fcon = space['config']
    param = space['param']
    fcon.model.init_params = {'a': -param, 'b':param}
    model = RNNModel(fcon.model)
    optimizer = fcon.train.get_optimizer(model.parameters())
    train_loss, val_loss = train_model(fcon, model, optimizer, verbose = False, save_interval = 3)
    return {
        'loss': train_loss[-1],
        'status': STATUS_OK,
        'eval_time': time.time(),
        'val_loss': val_loss
    }

In [ ]:
for model_type in ['lstm', 'gru']:
    for hidden_size in [64, 128, 256, 512]:
        print('Hidden Size: {}'.format(hidden_size))
        trials = Trials()
    #     trials = pkl.load(open('{}_trials_{}.p'.format(fcon.model.model_type, hidden_size), 'rb'))
        mcon = ModelConfig(model_type, 1, hidden_size, dcon.input_size, dropout, 'uni', {'a': -uni_param, 'b':uni_param}, device, bias = False, id_init_param = 'b')
        tcon = TrainConfig('Models', batch_size, max_epoch, 'adam', learning_rate, {}, scheduler = sch, scheduler_params= sch_params, start_epoch = 0)
        fcon = FullConfig(dcon, tcon, mcon)

        space = hp.choice('uni_param',[
            {
                'param': hp.quniform('param', .04, .24, .001),
                'config': fcon
            }    
        ])


        model = RNNModel(mcon).to(mcon.device)
        optimizer = tcon.get_optimizer(model.parameters())
        best = fmin(objective, 
                    space = space,
                    algo = tpe.suggest,
                    trials = trials,
                    max_evals = 60
                   )
        pkl.dump(trials, open('{}_trials_{}.p'.format(fcon.model.model_type, hidden_size), 'wb'))

Hidden Size: 64
100%|██████████| 60/60 [55:06<00:00, 55.12s/trial, best loss: 1.9232112938165664]
Hidden Size: 128
100%|██████████| 60/60 [1:19:29<00:00, 79.49s/trial, best loss: 1.7544687151908875]
Hidden Size: 256
100%|██████████| 60/60 [2:04:22<00:00, 124.37s/trial, best loss: 1.530445828437805]  
Hidden Size: 512
100%|██████████| 60/60 [4:18:04<00:00, 258.08s/trial, best loss: 1.3131837475299835]  
Hidden Size: 64
100%|██████████| 60/60 [53:15<00:00, 53.25s/trial, best loss: 1.7973689597845077]
Hidden Size: 128
100%|██████████| 60/60 [1:06:41<00:00, 66.69s/trial, best loss: 1.6492356532812118]
Hidden Size: 256
100%|██████████| 60/60 [1:50:28<00:00, 110.47s/trial, best loss: 1.4638451838493347]
Hidden Size: 512
  2%|▏         | 1/60 [03:42<3:38:25, 222.13s/trial, best loss: 1.3793333268165588]

# Load Trials and calculate Lyapunov Exponents

In [ ]:
hidden_size = 64
le_batch_size = 10
le_seq_length = 1000
warmup = 0
ON = 1

fcon.model.hidden_size = hidden_size
lcon = LyapConfig(le_batch_size, le_seq_length, ON_step = ON, warmup = warmup, one_hot = True)

In [ ]:
# trials = pkl.load(open('trials_{}.p'.format(hidden_size), 'rb'))

In [ ]:
epoch = 15
for hidden_size in [64, 128, 256, 512]:
    print(hidden_size)
    trials = pkl.load(open('{}_trials_{}.p'.format(fcon.model.model_type, hidden_size), 'rb'))
    fcon.model.rnn_atts['hidden_size'] = hidden_size
    lcon = LyapConfig(le_batch_size, le_seq_length, ON_step = ON, warmup = warmup, one_hot = True)
    data = lcon.get_input(fcon)
    for trial in trials:
        uni_param = trial['misc']['vals']['param'][0]
        fcon.model.init_params = {'a': -uni_param, 'b':uni_param}
        ckpt = load_checkpoint(fcon, epoch)
        model = ckpt[0].to(fcon.device)
        LE_stats, _ = lcon.calc_lyap(data, model, fcon)
        torch.save(LE_stats, 'LE_stats/{}_LE_stats_e{}.p'.format(fcon.name(), epoch))

In [ ]:
for hidden_size in [64, 128, 256, 512]:
    fcon.model.hidden_size = hidden_size
    lcon = LyapConfig(le_batch_size, le_seq_length, ON_step = ON, warmup = warmup, one_hot = True)

In [ ]:
fcon.model.model_type = 'lstm'
for hidden_size in [64, 128, 256, 512]:
    fcon.model.rnn_atts['hidden_size'] = hidden_size
    trials = pkl.load(open('{}_trials_{}.p'.format(fcon.model.model_type, hidden_size), 'rb'))
    LE_tensor = torch.zeros((30, hidden_size))
    LE_stds = torch.zeros((30, hidden_size))
    for i, trial in enumerate(trials):
        uni_param = trial['misc']['vals']['param'][0]
        fcon.model.init_params = {'a': -uni_param, 'b':uni_param}
        LE_tensor[i, :], LE_stds[i, :] = torch.load('LE_stats/{}_LE_stats_e{}.p'.format(fcon.name(), 15))
    torch.save(LE_tensor, 'LE_stats/{}_{}_LEs.p'.format(fcon.model.model_type, hidden_size))
    torch.save(LE_stds, 'LE_stats/{}_{}_LEerrors.p'.format(fcon.model.model_type, hidden_size))

In [ ]:
fcon.model.model_type = 'gru'
torch.load('LE_stats/{}_LE_stats_e{}.p'.format(fcon.name(), 15))[0].shape

In [ ]:
fcon.model.model_type = 'gru'
for hidden_size in [64, 128, 256, 512]:
    fcon.model.rnn_atts['hidden_size'] = hidden_size
    trials = pkl.load(open('{}_trials_{}.p'.format(fcon.model.model_type, hidden_size), 'rb'))
    final_tloss = torch.zeros((30))
    vlosses = torch.zeros((30))
    for i, trial in enumerate(trials):
        uni_param = trial['misc']['vals']['param'][0]
        fcon.model.init_params = {'a': -uni_param, 'b':uni_param}
        _, _, train_loss, val_loss = load_checkpoint(fcon, fcon.train.max_epoch)
        final_tloss[i] = train_loss[-1]
        vlosses[i] = val_loss
    torch.save(final_tloss, 'LE_stats/{}_{}_trainLoss.p'.format(fcon.model.model_type, hidden_size))
    torch.save(vlosses, 'LE_stats/{}_{}_valLoss.p'.format(fcon.model.model_type, hidden_size))

In [ ]:
vlosses

In [ ]:
plt.plot(vlosses)

In [ ]:
model_type = 'lstm'
for hidden_size in [64, 128, 256, 512]:
    fcon.model.rnn_atts['hidden_size'] = hidden_size
    fcon.model.model_type = model_type
    LEs = torch.load('LE_stats/{}_{}_LEs.p'.format(fcon.model.model_type, hidden_size))
    LE_errors = torch.load('LE_stats/{}_{}_LEerrors.p'.format(fcon.model.model_type, hidden_size))
    train_loss = torch.load('LE_stats/{}_{}_trainLoss.p'.format(fcon.model.model_type, hidden_size))
    val_loss = torch.load('LE_stats/{}_{}_valLoss.p'.format(fcon.model.model_type, hidden_size))
    plot_summary(val_loss, [0,1,4,2, 9, 21], LEs, LE_errors)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
def plot_summary(val_losses, idcs, LEs, LE_errors):
    plt.rc('font', family = 'serif')
    plt.rc('xtick', labelsize = 'medium')
    plt.rc('ytick', labelsize = 'medium')
    fontsize = 12
    hidden_size = LEs.shape[1]
    fig= plt.figure(figsize = (6,3))
    ax1 = fig.add_subplot(1, 2, 1)
    # ax2 = fig.add_subplot(1, 2, 2)
    ax_max = fig.add_subplot(1, 2, 2)



    idcs = torch.LongTensor(idcs)
#     print(idcs)
    val_lims = [torch.min(val_losses), torch.max(val_losses)]
    plt.rc('ytick', labelsize = 'small')
    plt.rc('xtick', labelsize = 'small')
    axin1 = ax1.inset_axes([0.15, 0.1, 0.38, 0.38])

    for i, idx in enumerate(idcs):
        axin1.plot(range(0,10), LEs[idx, :10], '-o', markersize = 2.5, linewidth = .5, label = idx.item())
        axin1.set_yticks([0, -.5])
    #     axin1.set_ylim([-., 0])
    axin1.plot([0, 10], [0, 0], 'white', linewidth = .5)

    axin1.set_xlim([0,9])
    axin1.yaxis.set_minor_locator(MultipleLocator(.1))
    #     axin1.set_xlabel(i)
    plt.rc('ytick', labelsize = 'medium')
    plt.rc('xtick', labelsize = 'medium')
    for idx in range(len(val_losses)):
        ax1.scatter(range(0,hidden_size), LEs[idx, :], s = .5, color = 'grey', alpha = .1)
    for i, idx in enumerate(idcs):
        ax1.scatter(range(0,hidden_size), LEs[idx, :], s = 2, label = idx.item())
    # plt.plot([0, 512], [0, 0], 'k--')
    # ax1.set_title('(a)')
    ax1.set_ylabel(r'$\lambda_i$', fontsize = fontsize)
    ax1.set_xlabel(r'$i$', fontsize = fontsize)
    ax1.set_ylim([-8, 0])
    ax1.set_xlim([0, hidden_size])
    # plt.legend(title = 'Trial')
    # plt.savefig("Trial Spectra.png",bbox_inches="tight",dpi=400, format = 'png')


    x=  LEs[:, 0][idcs]
    y = np.array(val_losses)[idcs]
    ax_max.scatter(LEs[:,0], val_losses, color = 'gray', alpha = 0.4)
    for i in range(x.shape[0]):
        ax_max.scatter(x[i], y[i], s = 50)

    # ax_max.set_title('(b)')
    ax_max.set_xlabel(r' Max LE, $\lambda_{max}$', fontsize  =12)
    ax_max.set_ylabel('Validation Loss', fontsize = 12)
    # ax3.scatter(slopes1, val_loss_mins, s= 20, edgecolors = 'k', facecolors = 'none')
    # ax3.set_xlim([np.min(slopes1)-.0001, np.max(slopes1)+.0001]) 
    # ax3.set_xlabel('LE Slope', fontsize = fontsize)
    # ax3.set_ylabel('Validation Loss', fontsize = fontsize)

    # ax4.scatter(LE_avgs, val_loss_mins, s= 20, edgecolors = 'k', facecolors = 'none')
    # ax4.set_xlabel('LE Mean', fontsize = fontsize)
    # ax4.set_ylabel('Validation Loss', fontsize = fontsize)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    # ax4.savefig("Mean_ValMin.png",bbox_inches="tight",dpi=400, format = 'png')
#     plt.savefig("CharLE_Summary.png",bbox_inches="tight",dpi=400, format = 'png')

In [ ]:
torch.min(vlosses)